In [ ]:
import numpy as np
import ipywidgets as widgets
from ipywidgets import Widget, widget_serialization
from traitlets import Unicode, CInt, CFloat, CBool, Instance, List, Dict
from traittypes import Array

# Should be somewhere else
from unray.traits_numpy import array_serialization, shape_constraints

In [ ]:
# Setup some test data
coordinates = [
    [0,0,0],
    [1,0,0],
    [0,1,0],
    [.5,.5,1],
    ]

cells = [
    [0, 1, 2, 3],
    ]

indicators = [7]

values = [0.0, 0.2, 0.5, 1.0]

In [ ]:
# Generic visualization data widgets

# Maybe this corresponds to the bqplot Scale, LinearScale, ColorScale, ...?
@widgets.register('viswidgets.LUT')
class LUT(Widget):
    values = Array().tag(sync=True, **array_serialization)
    interpolation = Unicode("linear").tag(sync=True)
    #dtype, item_size, ...

@widgets.register('viswidgets.ScalarLUT')
class ScalarLUT(LUT):
    pass

@widgets.register('viswidgets.ColorLUT')
class ColorLUT(LUT):
    pass


@widgets.register('viswidgets.Mesh')
class Mesh(Widget):
    celltype = Unicode("tetrahedron").tag(sync=True)
    cells = Array().tag(sync=True, **array_serialization)
    coordinates = Array().tag(sync=True, **array_serialization)

@widgets.register('viswidgets.Field')
class Field(Widget):
    mesh = Instance(Mesh).tag(sync=True, **widget_serialization)
    space = Unicode("P1").tag(sync=True)
    values = Array().tag(sync=True, **array_serialization)
# Possibly ScalarField, VectorField

@widgets.register('viswidgets.NominalField')
class NominalField(Widget):
    mesh = Instance(Mesh).tag(sync=True, **widget_serialization)
    dim = CInt().tag(sync=True)
    values = Array().tag(sync=True, **array_serialization)


In [ ]:
# Somewhat generic plot widgets similar to bqplot classes here:  http://bqplot.readthedocs.io/en/stable/

@widgets.register('plotwidgets.Scale')
class Scale(Widget):
    pass
# LinearScale
# LogScale
# OrdinalScale
# ColorScale
# OrdinalColorScale

@widgets.register('plotwidgets.Axis')
class Axis(Widget):
    pass

@widgets.register('plotwidgets.Grid')
class Grid(Widget):
    axes = List(Instance(Axis)).tag(sync=True, **widget_serialization)

#class Mark(pythreejs.Group):
@widgets.register('plotwidgets.Mark')
class Mark(Widget):
    pass

#class Figure(pythreejs.Group):
@widgets.register('plotwidgets.Figure')
class Figure(Widget):
    grid = Instance(Grid).tag(sync=True, **widget_serialization)
    marks = List(Instance(Mark)).tag(sync=True, **widget_serialization)
    title = Unicode().tag(sync=True)


In [ ]:
# Unray specific plot widgets

class HasDensityField:
    density = Instance(Field).tag(sync=True, **widget_serialization)
    density_lut = Instance(ScalarLUT).tag(sync=True, **widget_serialization)

class HasColorField:
    color = Instance(Field).tag(sync=True, **widget_serialization)
    color_lut = Instance(ColorLUT).tag(sync=True, **widget_serialization)

class HasRestrictField:
    restrict = Instance(NominalField, allow_none=True).tag(sync=True, **widget_serialization)
    restrict_values = List(CInt()).tag(sync=True)

class HasWireframeField:
    wireframe = Dict(traits={
        "enable": CBool(False),
        "width": CFloat(0.01),
        "color": Unicode("black")
    }).tag(sync=True)

@widgets.register('unray.Surface')
class Surface(Mark): #, HasColorField, HasRestrictField, HasWireframeField):
    color = Instance(Field).tag(sync=True, **widget_serialization)
    color_lut = Instance(ColorLUT).tag(sync=True, **widget_serialization)
    restrict = Instance(NominalField, allow_none=True).tag(sync=True, **widget_serialization)
    restrict_values = List(CInt()).tag(sync=True)

@widgets.register('unray.XRay')
class XRay(Mark):#, HasDensityField, HasRestrictField):
    density = Instance(Field).tag(sync=True, **widget_serialization)
    density_lut = Instance(ScalarLUT).tag(sync=True, **widget_serialization)
    restrict = Instance(NominalField, allow_none=True).tag(sync=True, **widget_serialization)
    restrict_values = List(CInt()).tag(sync=True)


In [ ]:
# Example configuration of data widgets

# Unstructured mesh
mesh = Mesh(cells=cells, coordinates=coordinates)
#mesh.update(new_coordinates)

# Cellwise constant field over mesh
field0 = Field(mesh=mesh, values=values, space="P0")
#field0.update(new_values)

# Cellwise continuous linear field over mesh
field1 = Field(mesh=mesh, values=values, space="P1")
#field1.update(new_values)

# Cellwise discontinuous linear field over mesh
field2 = Field(mesh=mesh, values=values, space="D1")
#field2.update(new_values)

# Cell indicator values over mesh
ind3 = NominalField(mesh=mesh, values=indicators, dim=3)
#ind3.update(new_values)

# Facet indicator values over mesh
ind2 = NominalField(mesh=mesh, values=values, dim=2)
#ind2.update(new_values)

# Lookup tables
lut0 = ScalarLUT(values=[0.2, 0.4, 0.7, 1.0], interpolation="nearest")
lut1 = ScalarLUT(values=[0, 1], interpolation="linear")
lut2 = ColorLUT(values=[[1,0,0], [0,0,1]], interpolation="linear")

In [ ]:
# Example configuration of an unray plot

# Plot configurations can be packaged in dicts
surf_encoding = {
    #"color": { "field": field0, "lut": lut2 },
    #"color": { "value": "blue" },
    "color": field0,
    "color_lut": lut2,
    #"wireframe": { "enable": True, "width": 0.05, "color": "red" },
    #"restrict": { "indicators": ind3, "enabled_values": [3, 7] },
    #"restrict": ind3,
    #"restrict_values": [3, 7],
}
# But the encoding is really just passed as keyword arguments
surf = Surface(**surf_encoding)
xray = XRay(mesh=mesh, density=field1, density_lut=lut1)

In [ ]:
# Setup grid
axes = []
grid = Grid(*axes)
# three obj above subclasses of pythreejs.Object3D


# Setup figure
fig = Figure(
    grid=grid,
    marks=[surf, xray],
    title="Hello world",
    # Allow passing renderer and camera but create these by default if not provided:
    #scene=pythreejs.Scene(...),
    #renderer=pythreejs.Renderer(...),
    #camera=pythreejs.Camera(...)
    )
#display(fig)

In [ ]:
class A(HasTraits):
    d = Dict(trait=CInt(), traits={"a": CInt(), "b": Unicode()})
a = A()

In [ ]:
a.d = { "a": 3 }
a.d

In [ ]:
a.d = { "a": 3, "c": 4}
a.d

In [ ]:
from traitlets import HasTraits, TraitError, Int, Bool, validate

class Parity(HasTraits):
    value = Int()
    parity = Int()

    @validate('value')
    def _valid_value(self, proposal):
        print("value", proposal, proposal["owner"].parity, proposal["owner"].value)
        if proposal['value'] % 2 != self.parity:
            raise TraitError('value and parity should be consistent')
        return proposal['value']

    @validate('parity')
    def _valid_parity(self, proposal):
        print("parity", proposal, proposal["owner"].parity, proposal["owner"].value)
        parity = proposal['value']
        if parity not in [0, 1]:
            raise TraitError('parity should be 0 or 1')
        if self.value % 2 != parity:
            raise TraitError('value and parity should be consistent')
        return proposal['value']

parity_check = Parity(value=2)

# Changing required parity and value together while holding cross validation
with parity_check.hold_trait_notifications():
    parity_check.value = 1
    parity_check.parity = 3